In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tqdm import tqdm

In [2]:
# List all available devices detected by TensorFlow
print("Available devices:")
devices = tf.config.list_physical_devices()
for device in devices:
    print(device)

Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


## Load and prepare data

In [3]:
# Load the data
data = pd.read_csv("../dataset.csv")

In [4]:
# Separate features and target
X = data.drop("Label", axis=1)
y = data["Label"]

In [5]:
# Convert categorical data to numeric if necessary
X = pd.get_dummies(X, sparse=True)

In [6]:
# chunk_size = 10000  # Adjust based on your system's memory
# chunks = []
# for chunk in tqdm(pd.read_csv('../dataset.csv', chunksize=chunk_size)):
#     # Process each chunk
#     chunk = pd.get_dummies(chunk, sparse=True)
#     chunks.append(chunk)

# # Concatenate all processed chunks
# X = pd.concat(chunks, ignore_index=True)

In [7]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [8]:
# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:1059: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  if np.may_share_memory(array, array_orig):
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy arr

## Build an AutoEncoder

In [9]:
# Define the encoder
input_layer = Input(shape=(X_train_scaled.shape[1],))
encoded = Dense(128, activation="relu")(input_layer)
encoded = Dense(64, activation="relu")(encoded)

In [10]:
# Define the decoder
decoded = Dense(128, activation="relu")(encoded)
decoded = Dense(X_train_scaled.shape[1], activation="sigmoid")(decoded)

In [11]:
# Define the autoencoder
autoencoder = Model(input_layer, decoded)

# Encoder for predictions
encoder = Model(input_layer, encoded)

# Prediction model
predictor = Dense(1, activation="sigmoid")(encoded)
prediction_model = Model(input_layer, predictor)

# Compile models
autoencoder.compile(optimizer="adam", loss="mse")
prediction_model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

In [12]:
# Display the model summary
autoencoder.summary()
prediction_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 20090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,571,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20090)          │     2,591,610 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,179,834 (19.76 MB)

 Trainable params: 5,179,834 (19.76 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 20090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,571,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,579,969 (9.84 MB)

 Trainable params: 2,579,969 (9.84 MB)

 Non-trainable params: 0 (0.00 B)

## Train and evaluate

In [16]:
# Train the autoencoder
autoencoder.fit(
    X_train_scaled, X_train_scaled, epochs=30, batch_size=256, validation_split=0.2
)

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.8007 - val_loss: 0.8012
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.8007 - val_loss: 0.8011
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.8007 - val_loss: 0.8011
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.8007 - val_loss: 0.8011
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.8007 - val_loss: 0.8011
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.8007 - val_loss: 0.8011
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.8007 - val_loss: 0.8011
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.8007 - val_loss: 0.8010
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.8007 - val_loss: 0.8010
Epoch 10/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.8007 - val_loss: 0.8010
Epoch 11/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.8007 - val_loss: 0.8010
Epoch 12/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 0.8

In [17]:
# Train the prediction model
prediction_model.fit(
    X_train_scaled, y_train, epochs=30, batch_size=256, validation_split=0.2
)

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9993 - loss: 0.0013 - val_accuracy: 0.9887 - val_loss: 0.1474
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 1.0000 - loss: 7.9383e-05 - val_accuracy: 0.9875 - val_loss: 0.1482
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 6.8381e-05 - val_accuracy: 0.9875 - val_loss: 0.1475
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 6.8023e-05 - val_accuracy: 0.9875 - val_loss: 0.1468
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 6.5381e-05 - val_accuracy: 0.9875 - val_loss: 0.1461
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 6.4966e-05 - val_accuracy: 0.9875 - val_loss: 0.1456
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 4.9060e-05 - val_accuracy: 0.9881 - val_loss: 0.1450
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 4.8671e-0

In [18]:
# Evaluate the prediction model
test_loss, test_acc = prediction_model.evaluate(X_test_scaled, y_test)
print("Test Accuracy: {:.2f}%".format(test_acc * 100))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4809 - loss: 3.0862
Test Accuracy: 49.50%
